In [ ]:
#Petermann Glacier
peterPoints = [(-65.51712706815805,82.15009447647783),
(-65.51712706815805,78.38406522295739),
(-44.55698414800813,78.38406522295739),
(-44.55698414800813,82.15009447647783),
(-65.51712706815805,82.15009447647783)]

#Jakobshavn
jacobsPoints = [
(-51.85297729818772, 70.78302286892713),
(-51.85297729818772, 68.41440064014121),
(-46.50519742294812, 68.37754778777803),
(-46.50519742294812, 70.78302286892713),
(-51.85297729818772, 70.78302286892713) ]

#Antarctica
#Thwaites Glacier
thwaitesPoints = [
(-118.103998296609,-73.18220620018596),
(-118.103998296609,-76.27563309307305),
(-92.64141079610289,-76.27563309307305),
(-92.64141079610289,-73.18220620018596),
(-118.103998296609,-73.18220620018596) ]
#Drygalski

drygalskiPoints = [
(-62.2933314638862,-64.27767014248812),
(-62.2933314638862,-65.3068813457687),
(-58.81729523711281,-65.3068813457687),
(-58.81729523711281,-64.27767014248812),
(-62.2933314638862,-64.27767014248812)]

In [ ]:
import osgeo.ogr as ogr
import osgeo.osr as osr

import os

def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

def createShapeFile( name, points, proj ):
    driver = ogr.GetDriverByName("ESRI Shapefile")

    path = "ShapeFiles/{}_{}.shp".format( name, proj.shortName )
    ensure_dir(path)
    
    tmp_driver=ogr.GetDriverByName('MEMORY')
    tmp_source=tmp_driver.CreateDataSource('memData')
    
    # create the data source
    data_source = driver.CreateDataSource(path)

    # create the spatial reference, WGS84
    src = osr.SpatialReference()
    src.ImportFromEPSG(4326)
    
    target = osr.SpatialReference()
    target.ImportFromProj4(proj.proj4)

    targetprj = target
    transform = osr.CoordinateTransformation(src, targetprj)

    # create the layer
    tmp_layer = tmp_source.CreateLayer("landice", src, ogr.wkbPolygon)

    feature = ogr.Feature(tmp_layer.GetLayerDefn())
    #create the WKT for the feature using Python string formatting
    ring = ogr.Geometry(ogr.wkbLinearRing)
    
    for x,y in points:
        ring.AddPoint(x, y)
    
    poly = ogr.Geometry(ogr.wkbPolygon)
    poly.AddGeometry(ring)
    
    feature.SetGeometry( poly )
    # Create the feature in the layer (shapefile)
    
    outlayer = data_source.CreateLayer("landice", target,geom_type=ogr.wkbPolygon)

    # Add an ID field
    idField = ogr.FieldDefn("id", ogr.OFTInteger)
    outlayer.CreateField(idField)

    # Create the feature and set values
    transformed = feature.GetGeometryRef()
    transformed.Transform(transform)

    geom = ogr.CreateGeometryFromWkb(transformed.ExportToWkb())
    defn = outlayer.GetLayerDefn()
    feat = ogr.Feature(defn)
    feat.SetGeometry(geom)
    outlayer.CreateFeature(feat)
    
    # Dereference the feature
    feature = None
    tmp_layer = None
    outlayer = None
    data_source = None
    tmp_source = None
    

In [ ]:
from MalardClient.MalardClient import MalardClient
from MalardClient.DataSet import DataSet

client = MalardClient()

greenlandProjection = client.getProjection( DataSet("cryotempo", "GRIS_BaselineC_Q2", "greenland") )
antarcticProjection = client.getProjection( DataSet( "cryotempo", "AntarcticaV2b", "antarctic" ))

createShapeFile( "jakobshaven", jacobsPoints, greenlandProjection )
createShapeFile( "petermann", peterPoints, greenlandProjection )
createShapeFile( "thwaites", thwaitesPoints, antarcticProjection )
createShapeFile( "drygalski", drygalskiPoints, antarcticProjection )




In [ ]:
from MalardClient.MalardClient import MalardClient
from MalardClient.DataSet import DataSet
from MalardClient.MaskFilter import MaskFilter

from datetime import datetime

client = MalardClient()

greenland_ds = DataSet("cryotempo", "GRIS_BaselineC_Q2", "greenland")
greenland_oib = DataSet("cryotempo", "oib", "greenland")

shapeFile = "/home/jon/IdeaProjects/malard/python/cryotempo/ShapeFiles/jakobshaven_NPS.shp"
extentFilter=MaskFilter(p_shapeFile = shapeFile)

minT = datetime(2011,3,1,0,0)
maxT = datetime(2011,4,1,0,0)

#optional filters: Greenland columns below. 
#'lon', 'lat', 'elev', 'heading', 'demDiff', 'demDiffMad', 'demDiffMad2','phaseAmb', 'meanDiffSpread', 'wf_number', 'sampleNb', 'power',
#'powerdB', 'phase', 'phaseS', 'phaseSSegment', 'phaseConfidence', 'coh',
#'x', 'y', 'time', 'swathFileId', 'rowId

filters = [{"column":"coh","op":"gte","threshold":0.6},{"column":"power","op":"gt","threshold":10000.0}]

info = client.executeQueryPolygon( greenland_ds, minT, maxT, extentFilter=extentFilter, filters = filters )

infoDf = info.to_df

client.releaseCacheHandle(info.resultFileName)



In [ ]:
import matplotlib.pyplot as plt

def simplePlot(df):
## plot with no filters
    fig = plt.figure(figsize=(12,10))
    plt.scatter(x=df['x'], y=df['y'], c=df['elev'], marker='.', s=20, vmax=7500)
    plt.colorbar()
    plt.show()
    return fig

In [ ]:
simplePlot( infoDf )

In [ ]:
print( len(infoDf) )